In [ ]:
# Created by Diego Alvarez-Estevez, 2025

# This code is part of a Qiskit project.
#
# (C) Copyright IBM 2021, 2024.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

# Using IRIS dataset

# External imports
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd

from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit, StratifiedKFold

from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap

from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms.optimizers import SPSA

from qiskit_machine_learning.kernels import TrainableFidelityStatevectorKernel, FidelityStatevectorKernel # more efficient for statevector based simulators (better even than previous with Aer primitives)
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.utils.loss_functions import SVCLoss

from common_utils import *

# INITIALIZATION PARAMETERS
random_seed = 12345
algorithm_globals.random_seed = random_seed
num_repetitions = 30
num_QKT_iter = 400

scale_inputs = True
class_weight = None # None is default, assumes equeal class importance in binary setting, 

minimize_display = True

# PREPROCESSING DATSET
feature_dim = 4
poly_degree=7 #4 for n=2, 6-7 for n=4

iris = datasets.load_iris()
# Take all features
selected_features = range(feature_dim)
x_full = iris.data[:, selected_features]
y_full = iris.target

# Select problem version
linear_version = True
if linear_version:
    class_to_remove = 2 # Select two from [0,1,2]
    # select 2 to resemble https://pennylane.ai/qml/demos/tutorial_kernel_based_training/ -> setosa vs versicolor, linearly separable
    filename_output = 'results_IRIS_linear.pkl'
else:
    # non-linear version
    class_to_remove = 0 # Select two from [0,1,2]
    # select 0 versicolor vs virginica -> non-linearly separable (slightly)
    filename_output = 'results_IRIS_nonlinear.pkl'

# Convert to binary problem
y_full = y_full[np.where(iris.target != class_to_remove)]
x_full = x_full[np.where(iris.target != class_to_remove)]

# Transform labels to [1, -1]
min_value = np.min(y_full)
y_temp = y_full.copy()
y_temp[np.where(y_full == min_value)] = -1
y_temp[np.where(y_full != min_value)] = 1
y_full = y_temp.copy()

# Initialize structure to store benchmark results
results = {
    "LR": [],
    "SVM_linear": [],
    "SVM_poly": [],
    "SVM_rbf": [],
    "qSVM_ZZ": [],
    "qSVM_COV": [],
    "qSVM_ZZ_opt_s": [],
    "qSVM_COV_opt_s": [],
    "qSVM_ZZ_opt_d": [],
    "qSVM_COV_opt_d": []
}
results_hyper_params = {
    "LR": [],
    "SVM_linear": [],
    "SVM_poly": [],
    "SVM_rbf": [],
    "qSVM_ZZ": [],
    "qSVM_COV": []
}
results_loss = {
    "qSVM_ZZ_opt_s": [],
    "qSVM_COV_opt_s": [],
    "qSVM_ZZ_opt_d": [],
    "qSVM_COV_opt_d": []
}

# Set default verbose for GridSearchCV
used_verbose = 3
if minimize_display == True:
    used_verbose = 0

for rep in range(num_repetitions):
    print(f"REPETITION {rep+1} out of {num_repetitions}")
    
    # Random stratified sample using 70% for TR/VAL and 30% for TS
    xtrain_rep, xtest_rep, ytrain_rep, ytest_rep = train_test_split(x_full, y_full, test_size=0.3, random_state=random_seed, stratify=y_full)

    # Visualize data
    if not(minimize_display):
        print("xtrain_rep: ", xtrain_rep)
        print("ytrain_rep: ", ytrain_rep)
        print("xtest_rep: ", xtrain_rep)
        print("ytest_rep: ", ytrain_rep)
        
    print("Train set distribution:")
    count_labels(ytrain_rep)
    print("Test set distribution:")
    count_labels(ytest_rep)

    if random_seed is not None:
        random_seed += 1  # Increment seed to get different samples in each repetition

    ## ALL MODELS (CLASSICAL AND QUANTUM)
    
    # Define hyperparameter space
    C_range = np.logspace(-2, 2, 5);print("C_range:", C_range) # Returns numbers evenly space in log scale. Params (start, stop, num), with default base=10
    gamma_range = np.logspace(-3, 1, 5);print("Gamma_range:", gamma_range)

    cv = StratifiedKFold(n_splits=5, shuffle=False)

    ## CLASSICAL MODELS
       
    # Input scaling. For classical models standardize data. 
    # Note for this dataset features are in [0, 2PI] rage by default
    if scale_inputs == True:
        scaler = StandardScaler()
        xtrain = scaler.fit_transform(xtrain_rep)
        xtest = scaler.transform(xtest_rep)
    
    # CLASSICAL APPROACH (Logistic Regression) with hyperparameter search
    param_grid = dict(C=C_range)
    grid = GridSearchCV(LogisticRegression(class_weight=class_weight), param_grid=param_grid, cv=cv, verbose=used_verbose, refit=True) 
    
    start = time.time()
    grid.fit(xtrain, ytrain_rep) # Hence generating kind of train/val sets, keeping test data completely independent of this hyperparamter search
    elapsed = time.time() - start
    print(f"Classical LR hyperparameter search time: {round(elapsed,2)} seconds")
    print(
        "The best parameters are %s with a score of %0.2f"
        % (grid.best_params_, grid.best_score_)
    )
    ypred_tr = grid.predict(xtrain)
    print(f"Classical LR with BEST hyperparamters (acc train): {round(metrics.accuracy_score(ytrain_rep, ypred_tr),2)}, (bal_acc train): {round(metrics.balanced_accuracy_score(ytrain_rep, ypred_tr),2)}, (kappa train): {round(metrics.cohen_kappa_score(ytrain_rep, ypred_tr),2)}")
    ypred_ts = grid.predict(xtest)
    print(f"Classical LR with BEST hyperparamters (acc test): {round(metrics.accuracy_score(ytest_rep, ypred_ts),2)}, (bal_acc test): {round(metrics.balanced_accuracy_score(ytest_rep, ypred_ts),2)}, (kappa test): {round(metrics.cohen_kappa_score(ytest_rep, ypred_ts),2)}")
    # Obtain corresponding test metrics for final evaluation
    rep_metrics = compute_metrics(ytrain_rep, ypred_tr, ytest_rep, ypred_ts)
    results["LR"].append(rep_metrics)
    results_hyper_params["LR"].append(grid.best_params_)
    
    # CLASSICAL APPROACH (SVC, with linear kernel => no kernel trick) with hyperparameter search
    param_grid = dict(C=C_range)
    grid = GridSearchCV(SVC(kernel="linear", class_weight=class_weight), param_grid=param_grid, cv=cv, verbose=used_verbose, refit=True)

    start = time.time()
    grid.fit(xtrain, ytrain_rep) # Hence generating kind of train/val sets, keeping test data completely independent of this hyperparamter search
    elapsed = time.time() - start
    print(f"Classical SVM hyperparameter search time (linear): {round(elapsed,2)} seconds")
    print(
        "The best parameters are %s with a score of %0.2f"
        % (grid.best_params_, grid.best_score_)
    )
    ypred_tr = grid.predict(xtrain)
    print(f"Classical SVC -LINEAR kernel with BEST hyperparamters- (acc train): {round(metrics.accuracy_score(ytrain_rep, ypred_tr),2)}, (bal_acc train): {round(metrics.balanced_accuracy_score(ytrain_rep, ypred_tr),2)}, (kappa train): {round(metrics.cohen_kappa_score(ytrain_rep, ypred_tr),2)}")
    ypred_ts = grid.predict(xtest)
    print(f"Classical SVC -LINEAR kernel with BEST hyperparamters- (acc test): {round(metrics.accuracy_score(ytest_rep, ypred_ts),2)}, (bal_acc test): {round(metrics.balanced_accuracy_score(ytest_rep, ypred_ts),2)}, (kappa test): {round(metrics.cohen_kappa_score(ytest_rep, ypred_ts),2)}")
    # Obtain corresponding test metrics for final evaluation
    rep_metrics = compute_metrics(ytrain_rep, ypred_tr, ytest_rep, ypred_ts)
    results["SVM_linear"].append(rep_metrics)
    results_hyper_params["SVM_linear"].append(grid.best_params_)
    
    # CLASSICAL APPROACH (SVC, with polynomial kernel) with hyperparameter search
    param_grid = dict(C=C_range)
    grid = GridSearchCV(SVC(kernel="poly", degree=poly_degree, class_weight=class_weight), param_grid=param_grid, cv=cv, verbose=used_verbose, refit=True)

    start = time.time()
    grid.fit(xtrain, ytrain_rep) # Hence generating kind of train/val sets, keeping test data completely independent of this hyperparamter search
    elapsed = time.time() - start
    print(f"Classical SVM hyperparameter search time (poly): {round(elapsed,2)} seconds")
    print(
        "The best parameters are %s with a score of %0.2f"
        % (grid.best_params_, grid.best_score_)
    )
    ypred_tr = grid.predict(xtrain)
    print(f"Classical SVC -POLY kernel with BEST hyperparamters- (acc train): {round(metrics.accuracy_score(ytrain_rep, ypred_tr),2)}, (bal_acc train): {round(metrics.balanced_accuracy_score(ytrain_rep, ypred_tr),2)}, (kappa train): {round(metrics.cohen_kappa_score(ytrain_rep, ypred_tr),2)}")
    ypred_ts = grid.predict(xtest)
    print(f"Classical SVC -POLY kernel with BEST hyperparamters- (acc test): {round(metrics.accuracy_score(ytest_rep, ypred_ts),2)}, (bal_acc test): {round(metrics.balanced_accuracy_score(ytest_rep, ypred_ts),2)}, (kappa test): {round(metrics.cohen_kappa_score(ytest_rep, ypred_ts),2)}")
    # Obtain corresponding test metrics for final evaluation
    rep_metrics = compute_metrics(ytrain_rep, ypred_tr, ytest_rep, ypred_ts)
    results["SVM_poly"].append(rep_metrics)
    results_hyper_params["SVM_poly"].append(grid.best_params_)
    
    # CLASSICAL APPROACH (SVC, RBF) with hyperparameter search
    param_grid = dict(gamma=gamma_range, C=C_range)  
    grid = GridSearchCV(SVC(kernel="rbf", class_weight=class_weight), param_grid=param_grid, cv=cv, verbose=used_verbose, refit=True) 
    
    start = time.time()
    grid.fit(xtrain, ytrain_rep) # Hence generating kind of train/val sets, keeping test data completely independent of this hyperparamter search
    elapsed = time.time() - start
    print(f"Classical SVM hyperparameter search time (rbf): {round(elapsed,2)} seconds")
    print(
        "The best parameters are %s with a score of %0.2f"
        % (grid.best_params_, grid.best_score_)
    )
    ypred_tr = grid.predict(xtrain)
    print(f"Classical SVC -rbf kernel with BEST hyperparamters- (acc train): {round(metrics.accuracy_score(ytrain_rep, ypred_tr),2)}, (bal_acc train): {round(metrics.balanced_accuracy_score(ytrain_rep, ypred_tr),2)}, (kappa train): {round(metrics.cohen_kappa_score(ytrain_rep, ypred_tr),2)}")
    ypred_ts = grid.predict(xtest)
    print(f"Classical SVC -rbf kernel with BEST hyperparamters- (acc test): {round(metrics.accuracy_score(ytest_rep, ypred_ts),2)}, (bal_acc test): {round(metrics.balanced_accuracy_score(ytest_rep, ypred_ts),2)}, (kappa test): {round(metrics.cohen_kappa_score(ytest_rep, ypred_ts),2)}")
    # Obtain corresponding test metrics for final evaluation
    rep_metrics = compute_metrics(ytrain_rep, ypred_tr, ytest_rep, ypred_ts)
    results["SVM_rbf"].append(rep_metrics)
    results_hyper_params["SVM_rbf"].append(grid.best_params_)
    
    ## QUANTUM KERNEL METHODS
    
    # Input scaling. 
    # Note for this dataset features are in [0, 2PI] rage by default
    if scale_inputs == True:
        scaler = MinMaxScaler(feature_range=(0,2*np.pi))
        xtrain = scaler.fit_transform(xtrain_rep)
        xtest = scaler.transform(xtest_rep)

    # List of tested quantum feature maps
    quantum_feature_maps = ["Covariant", "ZZ"]
    
    for qfeat_map in quantum_feature_maps:
        print(f"\nQuantum Feature Map: {qfeat_map}")

        if qfeat_map == "Covariant":
            # Covariant Feature Map
            adhoc_feature_map = CovariantFeatureMap(feature_dim, reps=1)
        elif qfeat_map == "ZZ":
            # ZZFeatureMap
            adhoc_feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, insert_barriers=True)
        else:
            raise ValueError("Invalid quantum feature map")

        # Print chosen feature map
        if minimize_display == False:
            display(adhoc_feature_map.draw("mpl"))
            display(adhoc_feature_map.decompose().draw("mpl"))
        
        # QSVC hyperparameter search (C and "scaling factor / bandwidth")
        scaling_factor_range = [0.001, 0.01, 0.1, 0.5, 1.0];print("qKernel_scaling_factor_range:", scaling_factor_range)
        best_scaling_factor = None
        best_score = None
        for scaling_factor in scaling_factor_range:
            
            # Apply scaling factor (see "importance of kernel bandwith" paper)
            xtrain_scaled = xtrain * scaling_factor 
            xtest_scaled = xtest * scaling_factor

            print(f"\nQuantum Kernel with scaling factor: {scaling_factor} for range {scaling_factor_range}")
                
            # Computing (fidelity) kernel matrix entries
            # Using statevector simulation
            adhoc_kernel = FidelityStatevectorKernel(feature_map=adhoc_feature_map)
    
            # Calculate and plot resulting quantum kernel
            start = time.time()
            adhoc_matrix_train = adhoc_kernel.evaluate(x_vec=xtrain_scaled)
            elapsed = time.time() - start
            print(f"Quantum kernel matrix (training) time: {round(elapsed,2)} seconds")
            start = time.time()
            adhoc_matrix_test = adhoc_kernel.evaluate(x_vec=xtest_scaled, y_vec=xtrain_scaled)
            elapsed = time.time() - start
            print(f"Quantum kernel matrix (test) time: {round(elapsed,2)} seconds")
            
            if minimize_display == False:
                fig, axs = plt.subplots(1, 2, figsize=(10, 5))
                axs[0].imshow(
                    np.asmatrix(adhoc_matrix_train), interpolation="nearest", origin="upper", cmap="Blues"
                )
                axs[0].set_title("Ad hoc training kernel matrix")
                axs[1].imshow(np.asmatrix(adhoc_matrix_test), interpolation="nearest", origin="upper", cmap="Reds")
                axs[1].set_title("Ad hoc testing kernel matrix")
                plt.show()
    
            # Evaluate QSVC hyperparameter "C"
            param_grid = dict(C=C_range)  
            grid = GridSearchCV(SVC(kernel="precomputed", class_weight=class_weight), param_grid=param_grid, cv=cv, verbose=used_verbose, refit=True)
        
            start = time.time()
            grid.fit(adhoc_matrix_train, ytrain_rep) # Hence generating kind of train/val sets, keeping test data completely independent of this hyperparamter search
            elapsed = time.time() - start
            print(f"Quantum SVM hyperparameter search time (precomputed): {round(elapsed,2)} seconds")
            print(
                "The best parameters are %s with a score of %0.2f"
                % (grid.best_params_, grid.best_score_)
            )
            ypred_tr = grid.predict(adhoc_matrix_train)
            print(f"QSVC with BEST hyperparamters (acc train): {round(metrics.accuracy_score(ytrain_rep, ypred_tr),2)}, (bal_acc train): {round(metrics.balanced_accuracy_score(ytrain_rep, ypred_tr),2)}, (kappa train): {round(metrics.cohen_kappa_score(ytrain_rep, ypred_tr),2)}")
            ypred_ts = grid.predict(adhoc_matrix_test)
            print(f"QSVC with BEST hyperparamters (acc test): {round(metrics.accuracy_score(ytest_rep, ypred_ts),2)}, (bal_acc test): {round(metrics.balanced_accuracy_score(ytest_rep, ypred_ts),2)}, (kappa test): {round(metrics.cohen_kappa_score(ytest_rep, ypred_ts),2)}")
        
            # Evaluate "scaling_factor" hyperparameter
            current_score = grid.best_score_ # Accuracy score is default used within GridSearchCV
            if best_score == None:
                best_score = current_score
                best_scaling_factor = scaling_factor
            else:
                if current_score > best_score:
                    best_score = current_score
                    best_scaling_factor = scaling_factor
                    best_C = grid.best_params_["C"]
                    # Obtain corresponding test metrics for final evaluation
                    rep_metrics = compute_metrics(ytrain_rep, ypred_tr, ytest_rep, ypred_ts)

        # Store final results for this repetition
        if qfeat_map == "Covariant":
            # Covariant Feature Map
            results["qSVM_COV"].append(rep_metrics)
            results_hyper_params["qSVM_COV"].append({"C":best_C,"scaling_factor":best_scaling_factor})
        elif qfeat_map == "ZZ":
            # ZZFeatureMap
            results["qSVM_ZZ"].append(rep_metrics)
            results_hyper_params["qSVM_ZZ"].append({"C":best_C,"scaling_factor":best_scaling_factor})
        else:
            raise ValueError("Invalid quantum feature map")
                
        # QSVC + QKA (kernel training)

        # Set best found hyperparameters
        xtrain_scaled = xtrain * best_scaling_factor 
        xtest_scaled = xtest * best_scaling_factor
        
        print(f"\nTraining Quantum Kernel with best hypeparameters C: {best_C} and scaling factor: {best_scaling_factor}")
        
        for qkt_single_training_parameter in [True, False]:

            print(f"Shared configuration: {qkt_single_training_parameter}")
            
            # Create trainable feature map
            if qkt_single_training_parameter == True:
                training_params = ParameterVector("θ", 3)   
            else:
                training_params = ParameterVector("θ", adhoc_feature_map.num_qubits*3)
            #print(len(training_params))
            init_point = np.zeros(len(training_params)) # Initial point 0 => no rotation => original kernel. Change if other desired
            fm = TrainableFeatureMap(adhoc_feature_map, training_params, qkt_single_training_parameter)
        
            # Set up calculation of parameterized (fidelity) kernel matrix entries
            # Using statevector simulation
            quant_kernel = TrainableFidelityStatevectorKernel(feature_map=fm, training_parameters=training_params)
        
            # Set up the optimizer and loss function
            cb_qkt = QKTCallback()
            spsa_opt = SPSA(maxiter=num_QKT_iter, callback=cb_qkt.callback, blocking=True, second_order=True)
        
            # We provide best found C hyperparameter
            user_loss = SVCLoss(C=best_C, class_weight=class_weight)
        
            # Check correspondence with previous "baseline" value obtained (should match if init_point = 0, meaning no rotations applied) U(θ,0,0)=RY(θ)
            loss_value_baseline = user_loss.evaluate(init_point, quant_kernel, xtrain_scaled, ytrain_rep)
            print(f"QSVC associated (baseline trainable) train loss (using {user_loss} evaluate): {round(loss_value_baseline,4)}")
        
            # Instantiate a quantum kernel trainer
            qkt = QuantumKernelTrainer(
                quantum_kernel=quant_kernel, loss=user_loss, optimizer=spsa_opt, initial_point=init_point
            )
        
            # Train the kernel using QKT directly
            start = time.time()
            algorithm_globals.random_seed = random_seed
            qka_results = qkt.fit(xtrain_scaled, ytrain_rep)
            elapsed = time.time() - start
            print(f"QKT training time: {round(elapsed,2)} seconds")

            # Retrieve optimization results
            optimized_kernel = qka_results.quantum_kernel
            plot_data = cb_qkt.get_callback_data()  # callback data

            # Notice the "optimal_point" shown before corresponds with last evaluation point
            # WHICH MIGHT NOT CORRESPOND TO REAL optimization minimum!!!
            # Later in the code, the minimal (theoretical best) point is used
            
            if minimize_display == False:
                print(qka_results)
                
                # Calculate and show results corresponding to "end point"
                print(f"QKA[end_point], loss={round(plot_data[2][-1],4)}, params={plot_data[1][-1]}")
                
                # Use QSVC for classification
                qsvc = QSVC(quantum_kernel=optimized_kernel, C=best_C, class_weight=class_weight)
                
                # Fit the QSVC
                start = time.time()
                qsvc.fit(xtrain_scaled, ytrain_rep)
                elapsed = time.time() - start
                print(f"QSVC + QKA[end_point] optimized kernel training time: {round(elapsed,2)} seconds")
                
                ypred = qsvc.predict(xtrain_scaled)
                print(f"QSVC+QKA[end_point] (acc train): {round(metrics.accuracy_score(ytrain_rep, ypred),2)}, (bal_acc train): {round(metrics.balanced_accuracy_score(ytrain_rep, ypred),2)}, (kappa train): {round(metrics.cohen_kappa_score(ytrain_rep, ypred),2)}")
                ypred = qsvc.predict(xtest_scaled)
                print(f"QSVC+QKA[end_point] (acc test): {round(metrics.accuracy_score(ytest_rep, ypred),2)}, (bal_acc test): {round(metrics.balanced_accuracy_score(ytest_rep, ypred),2)}, (kappa test): {round(metrics.cohen_kappa_score(ytest_rep, ypred),2)}")
                
                K = optimized_kernel.evaluate(xtrain_scaled)  # kernel matrix evaluated on the training samples
                fig, ax = plt.subplots(1, 2, figsize=(14, 5))
                ax[0].plot([i + 1 for i in range(len(plot_data[0]))], np.array(plot_data[2]), c="k", marker="o")
                ax[0].set_xlabel("Iterations")
                ax[0].set_ylabel("Loss")
                ax[1].imshow(np.asmatrix(K), interpolation="nearest", origin="upper", cmap="Blues")
                ax[1].set_title("Optimized training kernel matrix")
                #fig.tight_layout()
                plt.show()
            
            # Show performance values obtained by the theoretical (best) minimal point
            index_min = np.argmin(plot_data[2])
            min_loss_value = plot_data[2][index_min]
            print(f"QKA[minimal_point], loss={round(min_loss_value,4)}, params={plot_data[1][index_min]}")
            # Now train SVC with this configuration and check error
            quant_kernel.assign_training_parameters(plot_data[1][index_min])
            qsvc = QSVC(quantum_kernel=quant_kernel, C=best_C, class_weight=class_weight)

            # Store loss values for statistics
            loss_metrics = {
                'baseline': loss_value_baseline,
                'after_qkt': min_loss_value
            }
            
            # Fit the QSVC
            start = time.time()
            qsvc.fit(xtrain_scaled, ytrain_rep)
            elapsed = time.time() - start
            print(f"QSVC + QKA[minimal_point] optimized kernel training time: {round(elapsed,2)} seconds")
            
            # Evalaute accuracy
            ypred_tr = qsvc.predict(xtrain_scaled)
            print(f"QSVC+QKA[minimal_point] (acc train): {round(metrics.accuracy_score(ytrain_rep, ypred_tr),2)}, (bal_acc train): {round(metrics.balanced_accuracy_score(ytrain_rep, ypred_tr),2)}, (kappa train): {round(metrics.cohen_kappa_score(ytrain_rep, ypred_tr),2)}")
            ypred_ts = qsvc.predict(xtest_scaled)
            print(f"QSVC+QKA[minimal_point] (acc test): {round(metrics.accuracy_score(ytest_rep, ypred_ts),2)}, (bal_acc test): {round(metrics.balanced_accuracy_score(ytest_rep, ypred_ts),2)}, (kappa test): {round(metrics.cohen_kappa_score(ytest_rep, ypred_ts),2)}")
            
            # Print final kernel matrices
            if minimize_display == False:
                Kop_tr = quant_kernel.evaluate(xtrain_scaled)  # kernel matrix evaluated on the training samples
                Kop_ts = quant_kernel.evaluate(x_vec=xtest_scaled, y_vec=xtrain_scaled)  # kernel matrix evaluated on the test samples
                fig, axs = plt.subplots(1, 2, figsize=(10, 5))
                axs[0].imshow(np.asmatrix(Kop_tr), interpolation="nearest", origin="upper", cmap="Blues")
                axs[0].set_title("Final training kernel matrix")
                axs[1].imshow(np.asmatrix(Kop_ts), interpolation="nearest", origin="upper", cmap="Reds")
                axs[1].set_title("Final testing kernel matrix")
                plt.show()

            # Store final results for this repetition
            rep_metrics = compute_metrics(ytrain_rep, ypred_tr, ytest_rep, ypred_ts)
            
            if qfeat_map == "Covariant":
                if qkt_single_training_parameter == True:
                    results["qSVM_COV_opt_s"].append(rep_metrics)
                    results_loss["qSVM_COV_opt_s"].append(loss_metrics)
                else:
                    results["qSVM_COV_opt_d"].append(rep_metrics)
                    results_loss["qSVM_COV_opt_d"].append(loss_metrics)
            elif qfeat_map == "ZZ":
                if qkt_single_training_parameter == True:
                    results["qSVM_ZZ_opt_s"].append(rep_metrics)
                    results_loss["qSVM_ZZ_opt_s"].append(loss_metrics)
                else:
                    results["qSVM_ZZ_opt_d"].append(rep_metrics)
                    results_loss["qSVM_ZZ_opt_d"].append(loss_metrics)
            else:
                raise ValueError("Invalid quantum feature map")

    # Check point
    save_results(filename_output, [results, results_hyper_params, results_loss])

# END SCRIPT
